<a href="https://colab.research.google.com/github/HTG2021/ejemplo_booststrap/blob/main/teachablemachine_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from keras.models import load_model  # TensorFlow is required for Keras to work
#from tensorflow.keras.models import load_model
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Replace this with the path to your image
image = Image.open("reloj-test.jpg").convert("RGB")

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Class: Reloj
Confidence Score: 0.9824735


Convertir archivo modelo h5

In [6]:
import h5py
f = h5py.File("keras_Model.h5", mode="r+")
model_config_string = f.attrs.get("model_config")
if model_config_string.find('"groups": 1,') != -1:
    model_config_string = model_config_string.replace('"groups": 1,', '')
    f.attrs.modify('model_config', model_config_string)
    f.flush()
    model_config_string = f.attrs.get("model_config")
    assert model_config_string.find('"groups": 1,') == -1

f.close()

Subir archivo o usar webcam con el modelo

In [ ]:
from tensorflow.keras.models import load_model
from PIL import Image, ImageOps
import numpy as np
import cv2
from tkinter import Tk, filedialog

# Desactivar notación científica para mayor claridad
np.set_printoptions(suppress=True)

# Cargar el modelo
model = load_model("keras_Model.h5", compile=False)

# Cargar las etiquetas
class_names = open("labels.txt", "r").readlines()

# Crear el array con la forma correcta para alimentar al modelo
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)


def preprocess_image(image):
    """
    Preprocesar una imagen para que sea compatible con el modelo.
    """
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)
    image_array = np.asarray(image)
    normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1
    data[0] = normalized_image_array
    return data


def predict_image(data):
    """
    Realizar una predicción en la imagen preprocesada.
    """
    prediction = model.predict(data)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]
    print("Clase:", class_name[2:], end="")
    print("Confianza:", confidence_score)


def capture_with_webcam():
    """
    Capturar una imagen usando la webcam.
    """
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: No se pudo acceder a la cámara.")
        return

    print("Presiona 's' para capturar la imagen o 'q' para salir.")
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: No se pudo capturar el cuadro de video.")
            break

        cv2.imshow("Webcam - Presiona 's' para capturar", frame)

        # Esperar entrada del usuario
        key = cv2.waitKey(1)
        if key & 0xFF == ord('s'):  # Capturar imagen
            image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            cap.release()
            cv2.destroyAllWindows()
            return image
        elif key & 0xFF == ord('q'):  # Salir
            cap.release()
            cv2.destroyAllWindows()
            return None


def load_image_from_file():
    """
    Abrir un cuadro de diálogo para seleccionar una imagen.
    """
    root = Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(
        title="Seleccionar una imagen",
        filetypes=[("Archivos de imagen", "*.jpg;*.jpeg;*.png;*.bmp")]
    )
    if file_path:
        return Image.open(file_path).convert("RGB")
    else:
        print("No se seleccionó ninguna imagen.")
        return None


# Menú principal
print("Selecciona una opción:")
print("1. Subir una imagen")
print("2. Capturar con la webcam")
choice = input("Opción (1/2): ")

if choice == "1":
    image = load_image_from_file()
elif choice == "2":
    image = capture_with_webcam()
else:
    print("Opción no válida.")
    image = None

if image:
    data = preprocess_image(image)
    predict_image(data)
else:
    print("No se procesó ninguna imagen.")


Selecciona una opción:
1. Subir una imagen
2. Capturar con la webcam
Opción (1/2): 2
Error: No se pudo acceder a la cámara.
No se procesó ninguna imagen.


Webcam

In [ ]:
from keras.models import load_model  # TensorFlow is required for Keras to work
import cv2  # Install opencv-python
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# CAMERA can be 0 or 1 based on default camera of your computer
camera = cv2.VideoCapture(0)

while True:
    # Grab the webcamera's image.
    ret, image = camera.read()

    # Resize the raw image into (224-height,224-width) pixels
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Show the image in a window
    cv2.imshow("Webcam Image", image)

    # Make the image a numpy array and reshape it to the models input shape.
    image = np.asarray(image, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalize the image array
    image = (image / 127.5) - 1

    # Predicts the model
    prediction = model.predict(image)
    index = np.argmax(prediction)
    class_name = class_names[index]
    confidence_score = prediction[0][index]

    # Print prediction and confidence score
    print("Class:", class_name[2:], end="")
    print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

    # Listen to the keyboard for presses.
    keyboard_input = cv2.waitKey(1)

    # 27 is the ASCII for the esc key on your keyboard.
    if keyboard_input == 27:
        break

camera.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) /io/opencv/modules/imgproc/src/resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'resize'


In [ ]:
from keras.models import load_model  # TensorFlow is required for Keras to work
import cv2  # Install opencv-python
import numpy as np

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = load_model("keras_Model.h5", compile=False)

# Load the labels
class_names = open("labels.txt", "r").readlines()

# Initialize the camera (camera index could be 0 or 1 based on your system)
camera = cv2.VideoCapture(0)

# Check if the camera is opened correctly
if not camera.isOpened():
    print("Error: Could not open camera.")
    exit()

while True:
    # Grab a frame from the camera
    ret, image = camera.read()

    if not ret:
        print("Error: Failed to capture image")
        break

    # Resize the captured image to 224x224 pixels
    image_resized = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)

    # Show the resized image in a window
    cv2.imshow("Webcam Image", image_resized)

    # Convert the image to a numpy array and reshape to match model input shape
    image_array = np.asarray(image_resized, dtype=np.float32).reshape(1, 224, 224, 3)

    # Normalize the image
    image_array = (image_array / 127.5) - 1

    # Predict the class with the model
    prediction = model.predict(image_array)
    index = np.argmax(prediction)
    class_name = class_names[index].strip()  # Strip newlines or spaces from label
    confidence_score = prediction[0][index]

    # Print the predicted class and confidence score
    print("Class:", class_name)
    print("Confidence Score:", str(np.round(confidence_score * 100))[:-2], "%")

    # Wait for key press, and break the loop if 'Esc' is pressed (ASCII 27)
    keyboard_input = cv2.waitKey(1)

    if keyboard_input == 27:  # Esc key pressed
        print("Exiting...")
        break

# Release the camera and close OpenCV windows
camera.release()
cv2.destroyAllWindows()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Error: Could not open camera.
Error: Failed to capture image


error: OpenCV(4.10.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
